<a href="https://colab.research.google.com/github/Anil-matcha/langchain-tutorials/blob/main/LocalGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gpt4all langchain chromadb

In [2]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://uuki.live/")
data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

100%|██████████| 45.5M/45.5M [00:00<00:00, 320MiB/s]


Model downloaded at:  /root/.cache/gpt4all/ggml-all-MiniLM-L6-v2-f16.bin


In [4]:
question = "What is UUKI ?"
docs = vectorstore.similarity_search(question)[:2]

In [5]:
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

--2023-07-17 03:18:11--  https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin
Resolving gpt4all.io (gpt4all.io)... 172.67.71.169, 104.26.1.159, 104.26.0.159, ...
Connecting to gpt4all.io (gpt4all.io)|172.67.71.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785248281 (3.5G)
Saving to: ‘ggml-gpt4all-j-v1.3-groovy.bin’

ggml-gpt4all-j-v1.3 100%[===================>]   3.52G  39.8MB/s    in 75s     

2023-07-17 03:19:27 (48.5 MB/s) - ‘ggml-gpt4all-j-v1.3-groovy.bin’ saved [3785248281/3785248281]



In [6]:
from langchain.llms import GPT4All

llm = GPT4All(
    model="./ggml-gpt4all-j-v1.3-groovy.bin",
    max_tokens=2048,
)

Found model file at  ./ggml-gpt4all-j-v1.3-groovy.bin


In [7]:
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate

# Prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

# Chain
chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_CHAIN_PROMPT)

# Run
chain({"input_documents": docs, "question": question}, return_only_outputs=True)

{'output_text': " UUKI (Unified Universal Knowledge Infrastructure) is a web3-based community platform that empowers users to create their own content, build relationships with other members of the community, engage in discussions on various topics related to Web3 technology and products. It offers features such as live events, courses, coaching sessions, influencers' profiles, and one-time payment options for premium content creation. UUKI is designed specifically for creators, brands, and products built on web3 technologies like blockchain, decentralized applications (dApps), smart contracts, etc., making it an ideal platform to monetize their online presence or build a community around specific projects/products."}